In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

 # Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:

# Setup paths
base_dir = os.getenv("BASE_DIR")
scratch_dir = os.getenv("SCRATCH_DIR")

behav_dir = os.path.join(base_dir, "data", "behav")
brain_glmm_dir = os.path.join(scratch_dir, "output", "09_brain_content_glmm")

# Load data
df_2a = pd.read_csv(os.path.join(behav_dir, "prettymouth2a.csv"))

## Behavior data

In [24]:

df_2a = pd.read_csv(os.path.join(behav_dir, "prettymouth2a.csv"))
df_2b = pd.read_csv(os.path.join(behav_dir, "prettymouth2b.csv"))
df_2_postsurvey = pd.read_csv(os.path.join(behav_dir, "prettymouth2_postsurvey.csv")).iloc[2:]

df_2a_response = df_2a.dropna(subset=["key_resp_2.rt"])
df_2b_response = df_2b.dropna(subset=["key_resp_2.rt"])

common_participants_2a = set(df_2a_response["participant"]).intersection(df_2_postsurvey["participant"])
common_participants_2b = set(df_2b_response["participant"]).intersection(df_2_postsurvey["participant"])

survey_df = df_2_postsurvey[df_2_postsurvey["participant"].isin(common_participants_2a.union(common_participants_2b))]


In [28]:
len(common_participants_2a), len(common_participants_2b)

(63, 59)

In [26]:
survey_df['gender'].value_counts()

gender
Male                         62
Female                       57
Non-binary / third gender     3
Name: count, dtype: int64

In [27]:
survey_df['age'].value_counts()

age
25-34 years old    46
35-44 years old    29
18-24 years old    18
55-64 years old    14
45-54 years old    12
65+ years old       3
Name: count, dtype: int64

## Brain glmm

In [3]:
# Create main effects table
def create_main_effects_table(data):
    # Initialize lists to store results
    features = []
    coefficients = []
    posterior_sds = []
    prob_positive = []
    bayes_factors = []
    evidence_categories = []
    odds_ratios = []
    odds_lower = []
    odds_upper = []
    
    # Extract feature results from main analysis
    feature_results = data['main_analysis']['feature_results']
    
    # Iterate through each feature
    for feature_name, results in feature_results.items():
        # Add feature main effect
        features.append(f"{feature_name}")
        coefficients.append(results['coefficients'][feature_name])
        posterior_sds.append(results['posterior_sds'][feature_name])
        prob_positive.append(results['prob_positive'][feature_name])
        bayes_factors.append(results['bayes_factors'][feature_name])
        evidence_categories.append(results['evidence_categories'][feature_name])
        odds_ratios.append(results['odds_ratios'][feature_name]['odds_ratio'])
        odds_lower.append(results['odds_ratios'][feature_name]['lower'])
        odds_upper.append(results['odds_ratios'][feature_name]['upper'])
        
        # Add group interaction effect
        interaction_key = f"group_{feature_name}_interaction"
        features.append(f"{feature_name}:Group Interaction")
        coefficients.append(results['coefficients'][interaction_key])
        posterior_sds.append(results['posterior_sds'][interaction_key])
        prob_positive.append(results['prob_positive'][interaction_key])
        bayes_factors.append(results['bayes_factors'][interaction_key])
        evidence_categories.append(results['evidence_categories'][interaction_key])
        odds_ratios.append(results['odds_ratios'][interaction_key]['odds_ratio'])
        odds_lower.append(results['odds_ratios'][interaction_key]['lower'])
        odds_upper.append(results['odds_ratios'][interaction_key]['upper'])
    
    # Create DataFrame
    df = pd.DataFrame({
        'Feature': features,
        'Coefficient': coefficients,
        'Std. Error': posterior_sds,
        'P(Effect > 0)': prob_positive,
        'Bayes Factor': bayes_factors,
        'Evidence Category': evidence_categories,
        'Odds Ratio': odds_ratios,
        'OR Lower': odds_lower,
        'OR Upper': odds_upper
    })
    
    # Sort by Bayes Factor (descending)
    df = df.sort_values(by='Bayes Factor', ascending=False)
    
    # Format numeric columns
    df['Coefficient'] = df['Coefficient'].round(3)
    df['Std. Error'] = df['Std. Error'].round(3)
    df['P(Effect > 0)'] = df['P(Effect > 0)'].round(3)
    df['Bayes Factor'] = df['Bayes Factor'].round(2)
    df['Odds Ratio'] = df['Odds Ratio'].round(3)
    df['OR Lower'] = df['OR Lower'].round(3)
    df['OR Upper'] = df['OR Upper'].round(3)
    
    return df

# Create group-specific effects table
def create_group_effects_table(data):
    # Initialize lists to store results
    features = []
    affair_coefs = []
    affair_ors = []
    affair_prob_pos = []
    paranoia_coefs = []
    paranoia_ors = []
    paranoia_prob_pos = []
    diff_coefs = []
    prob_stronger_affair = []
    
    # Extract feature results from main analysis
    feature_results = data['main_analysis']['feature_results']
    
    # Iterate through each feature
    for feature_name, results in feature_results.items():
        # Check if group_specific_effects exists for this feature
        if 'group_specific_effects' in results and feature_name in results['group_specific_effects']:
            gse = results['group_specific_effects'][feature_name]
            
            features.append(feature_name)
            
            # Affair group
            affair_coefs.append(gse['affair_group']['coefficient'])
            affair_ors.append(gse['affair_group']['odds_ratio'])
            affair_prob_pos.append(gse['affair_group']['prob_positive'])
            
            # Paranoia group
            paranoia_coefs.append(gse['paranoia_group']['coefficient'])
            paranoia_ors.append(gse['paranoia_group']['odds_ratio'])
            paranoia_prob_pos.append(gse['paranoia_group']['prob_positive'])
            
            # Difference between groups
            diff_coefs.append(gse['diff_between_groups']['coefficient'])
            prob_stronger_affair.append(gse['diff_between_groups']['prob_stronger_in_affair'])
    
    # Create DataFrame
    df = pd.DataFrame({
        'Feature': features,
        'Affair Coef': affair_coefs,
        'Affair OR': affair_ors,
        'Affair P(>0)': affair_prob_pos,
        'Paranoia Coef': paranoia_coefs,
        'Paranoia OR': paranoia_ors,
        'Paranoia P(>0)': paranoia_prob_pos,
        'Diff (A-P)': diff_coefs,
        'P(Stronger in Affair)': prob_stronger_affair
    })
    
    # Format numeric columns
    df['Affair Coef'] = df['Affair Coef'].round(3)
    df['Affair OR'] = df['Affair OR'].round(3)
    df['Affair P(>0)'] = df['Affair P(>0)'].round(3)
    df['Paranoia Coef'] = df['Paranoia Coef'].round(3)
    df['Paranoia OR'] = df['Paranoia OR'].round(3)
    df['Paranoia P(>0)'] = df['Paranoia P(>0)'].round(3)
    df['Diff (A-P)'] = df['Diff (A-P)'].round(3)
    df['P(Stronger in Affair)'] = df['P(Stronger in Affair)'].round(3)
    
    # Sort by absolute difference magnitude
    df = df.sort_values(by='Diff (A-P)', key=abs, ascending=False)
    
    return df

# Create multiple comparison table
def create_multiple_comparison_table(data):
    # Extract multiple comparison results
    mc_results = data['main_analysis']['multiple_comparison']
    fdr_threshold = mc_results['fdr_threshold']
    credible_effects = mc_results['credible_effects']
    all_effects = mc_results['all_effects']
    
    # Create DataFrame
    effects = []
    posterior_probs = []
    fdrs = []
    is_credible = []
    
    for effect, values in all_effects.items():
        effects.append(effect)
        posterior_probs.append(values['posterior_prob'])
        fdrs.append(values['fdr'])
        is_credible.append(effect in credible_effects)
    
    df = pd.DataFrame({
        'Effect': effects,
        'Posterior Probability': posterior_probs,
        'FDR': fdrs,
        'Credible (FDR < 0.05)': is_credible
    })
    
    # Sort by FDR (ascending)
    df = df.sort_values(by='FDR')
    
    # Format numeric columns
    df['Posterior Probability'] = df['Posterior Probability'].round(3)
    df['FDR'] = df['FDR'].round(3)
    
    return df

# Function to create a summary of cross-validation results
def create_cv_summary(data):
    cv_results = data['cross_validation']['subject_cv']
    
    # Group by group
    affair_results = [r for r in cv_results if r['group'] == 'affair']
    paranoia_results = [r for r in cv_results if r['group'] == 'paranoia']
    
    # Extract coefficients for each interaction term
    def extract_interaction_coeffs(results, interaction_name):
        return [r['coefficients'].get(f'group_{interaction_name}_interaction', np.nan) for r in results]
    
    # Get all interaction names (excluding 'const' and 'group_coded')
    sample_coeffs = cv_results[0]['coefficients']
    interaction_names = [k.replace('group_', '').replace('_interaction', '') for k in sample_coeffs.keys() 
                         if k not in ['const', 'group_coded'] and '_interaction' in k]
    
    # Create DataFrames for each group
    df_affair = pd.DataFrame({
        'Subject': [r['subject'] for r in affair_results],
        **{name: extract_interaction_coeffs(affair_results, name) for name in interaction_names}
    })
    
    df_paranoia = pd.DataFrame({
        'Subject': [r['subject'] for r in paranoia_results],
        **{name: extract_interaction_coeffs(paranoia_results, name) for name in interaction_names}
    })
    
    # Calculate summary statistics
    summary_stats = []
    for name in interaction_names:
        affair_mean = df_affair[name].mean()
        affair_std = df_affair[name].std()
        paranoia_mean = df_paranoia[name].mean()
        paranoia_std = df_paranoia[name].std()
        
        summary_stats.append({
            'Interaction': name,
            'Affair Mean': affair_mean,
            'Affair Std': affair_std,
            'Paranoia Mean': paranoia_mean,
            'Paranoia Std': paranoia_std,
            'Stability Ratio': min(affair_std, paranoia_std) / max(affair_std, paranoia_std)
        })
    
    df_summary = pd.DataFrame(summary_stats)
    
    # Format numeric columns
    for col in df_summary.columns:
        if col != 'Interaction':
            df_summary[col] = df_summary[col].round(4)
    
    # Sort by stability ratio (descending)
    df_summary = df_summary.sort_values(by='Stability Ratio', ascending=False)
    
    return df_summary

# Create all tables from the data
def create_all_tables(data):
    # Create tables
    main_effects_table = create_main_effects_table(data)
    group_effects_table = create_group_effects_table(data)
    multiple_comparison_table = create_multiple_comparison_table(data)
    cv_summary_table = create_cv_summary(data)
    
    # Group occupancy and transition statistics
    group_stats = data['main_analysis']['group_stats']
    
    occupancy_df = pd.DataFrame({
        'Group': ['Affair', 'Paranoia'],
        'Occupancy Rate': [group_stats['occupancy']['affair'], group_stats['occupancy']['paranoia']]
    })
    
    transition_df = pd.DataFrame({
        'Group': ['Affair', 'Paranoia'],
        'Entry Rate': [group_stats['transitions']['affair']['entry_rate'], 
                      group_stats['transitions']['paranoia']['entry_rate']],
        'Exit Rate': [group_stats['transitions']['affair']['exit_rate'], 
                     group_stats['transitions']['paranoia']['exit_rate']]
    })
    
    return {
        'main_effects': main_effects_table,
        'group_effects': group_effects_table,
        'multiple_comparison': multiple_comparison_table,
        'cv_summary': cv_summary_table,
        'occupancy': occupancy_df,
        'transitions': transition_df
    }

# Create visualizations
def create_visualizations(tables):
    # 1. Create forest plot of main effects and interactions
    plt.figure(figsize=(12, 10))
    
    # Filter to effects with Bayes Factor > 1
    plot_data = tables['main_effects'][tables['main_effects']['Bayes Factor'] > 1].copy()
    plot_data = plot_data.sort_values('Bayes Factor', ascending=True)
    
    # Create forest plot
    y_pos = range(len(plot_data))
    
    plt.errorbar(
        x=plot_data['Odds Ratio'],
        y=y_pos,
        xerr=[plot_data['Odds Ratio'] - plot_data['OR Lower'], 
              plot_data['OR Upper'] - plot_data['Odds Ratio']],
        fmt='o',
        capsize=5,
        elinewidth=1,
        markeredgewidth=1,
        markersize=8
    )
    
    # Add vertical line at OR=1 (no effect)
    plt.axvline(x=1, color='black', linestyle='--', alpha=0.7)
    
    # Customize plot
    plt.yticks(y_pos, plot_data['Feature'])
    plt.xlabel('Odds Ratio (95% HDI)')
    plt.title('Forest Plot of Effects')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    
    # 2. Create heatmap of group-specific effects
    plt.figure(figsize=(10, 8))
    
    # Prepare data for heatmap (difference between groups for each feature)
    heatmap_data = tables['group_effects'][['Feature', 'Diff (A-P)']].copy()
    heatmap_data = heatmap_data.sort_values('Diff (A-P)', key=abs, ascending=False)
    
    # Reshape for heatmap
    heatmap_matrix = pd.DataFrame(index=heatmap_data['Feature'], data={
        'Group Difference (A-P)': heatmap_data['Diff (A-P)']
    })
    
    # Create heatmap
    sns.heatmap(
        heatmap_matrix, 
        cmap='RdBu_r', 
        center=0,
        annot=True, 
        fmt=".3f", 
        linewidths=.5,
        cbar_kws={'label': 'Effect Size Difference (Affair - Paranoia)'}
    )
    
    plt.title('Difference in Effect Size Between Groups')
    plt.tight_layout()
    
    # Return the visualization figures
    return plt.figure(1), plt.figure(2)

In [5]:
def main(data):
    """
    Generate comprehensive tables and visualizations for neural state dynamics analysis.
    
    Parameters:
    -----------
    data_path : str
        Path to the data file (default: 'paste.txt')
    
    Returns:
    --------
    dict
        Dictionary containing all generated tables
    """
    # Load data and create tables
    tables = create_all_tables(data)
    
    # Print tables with meaningful headers
    print("\n=== NEURAL STATE DYNAMICS ANALYSIS RESULTS ===\n")
    
    print("TABLE 1: MAIN EFFECTS AND INTERACTIONS")
    print("----------------------------------------")
    print(tables['main_effects'].to_string(index=False))
    
    print("\n\nTABLE 2: GROUP-SPECIFIC EFFECTS")
    print("--------------------------------")
    print(tables['group_effects'].to_string(index=False))
    
    print("\n\nTABLE 3: MULTIPLE COMPARISONS ANALYSIS")
    print("-------------------------------------")
    print(tables['multiple_comparison'].to_string(index=False))
    
    print("\n\nTABLE 4: CROSS-VALIDATION SUMMARY")
    print("---------------------------------")
    print(tables['cv_summary'].to_string(index=False))
    
    print("\n\nTABLE 5: STATE OCCUPANCY RATES")
    print("-----------------------------")
    print(tables['occupancy'].to_string(index=False))
    
    print("\n\nTABLE 6: STATE TRANSITION RATES")
    print("------------------------------")
    print(tables['transitions'].to_string(index=False))
    
    # Create and save visualizations
    forest_plot, heatmap_plot = create_visualizations(tables)
    
    # Additional summary information
    print("\n\nKEY FINDINGS SUMMARY:")
    print("--------------------")
    
    # Find strongest evidence features (BF > 10)
    strong_evidence = tables['main_effects'][tables['main_effects']['Bayes Factor'] > 10]
    if not strong_evidence.empty:
        print(f"Features with strong evidence (BF > 10): {', '.join(strong_evidence['Feature'].tolist())}")
    
    # Find significant group differences (FDR < 0.05)
    sig_diff = tables['multiple_comparison'][tables['multiple_comparison']['Credible (FDR < 0.05)'] == True]
    if not sig_diff.empty:
        print(f"Credible group differences (FDR < 0.05): {', '.join(sig_diff['Effect'].tolist())}")
    
    # Group occupancy difference
    occ_diff = tables['occupancy']['Occupancy Rate'].iloc[0] - tables['occupancy']['Occupancy Rate'].iloc[1]
    print(f"State occupancy difference (Affair - Paranoia): {occ_diff:.4f}")
    
    return tables

# Function to create customized analysis for neural state dynamics paper
def create_paper_tables(data):
    """
    Generate publication-ready tables for a neural state dynamics paper.
    
    Parameters:
    -----------
    data_path : str
        Path to the data file
        
    Returns:
    --------
    dict
        Dictionary containing formatted publication tables
    """
    tables = create_all_tables(data)
    
    # Table 1: Main Effects (Publication Format)
    pub_main_effects = tables['main_effects'][['Feature', 'Coefficient', 'Std. Error', 
                                              'P(Effect > 0)', 'Bayes Factor', 'Odds Ratio']].copy()
    
    # Add significance markers based on Bayes Factor
    def get_significance(bf):
        if bf >= 100:
            return "***"
        elif bf >= 10:
            return "**"
        elif bf >= 3:
            return "*"
        else:
            return ""
    
    pub_main_effects['Significance'] = pub_main_effects['Bayes Factor'].apply(get_significance)
    pub_main_effects['Coef ± SE'] = pub_main_effects.apply(
        lambda row: f"{row['Coefficient']} ± {row['Std. Error']} {row['Significance']}", axis=1)
    
    pub_main_effects = pub_main_effects[['Feature', 'Coef ± SE', 'P(Effect > 0)', 'Bayes Factor', 'Odds Ratio']]
    
    # Table 2: Group Comparison (Publication Format)
    pub_group_effects = tables['group_effects'][['Feature', 'Affair Coef', 'Paranoia Coef', 
                                                'Diff (A-P)', 'P(Stronger in Affair)']].copy()
    
    # Mark credible differences
    credible_effects = tables['multiple_comparison'][tables['multiple_comparison']['Credible (FDR < 0.05)'] == True]['Effect']
    pub_group_effects['Credible Diff'] = pub_group_effects['Feature'].apply(
        lambda x: any(x in effect for effect in credible_effects))
    
    # Format group differences
    pub_group_effects['Group Difference'] = pub_group_effects.apply(
        lambda row: f"{row['Diff (A-P)']}{' †' if row['Credible Diff'] else ''}", axis=1)
    
    pub_group_effects = pub_group_effects[['Feature', 'Affair Coef', 'Paranoia Coef', 
                                          'Group Difference', 'P(Stronger in Affair)']]
    
    # Create the dictionary of publication tables
    pub_tables = {
        'main_effects': pub_main_effects,
        'group_effects': pub_group_effects,
        'occupancy': tables['occupancy'],
        'transitions': tables['transitions'],
    }
    
    return pub_tables


### Cluster 1

In [17]:
cluster_id = 1
model_id = 2
file = os.path.join(brain_glmm_dir, f"cluster{cluster_id}_combined_{model_id}states_deviation_th085", "complete_results.npz")
results = np.load(file, allow_pickle=True)["results"]
main(results.item())


=== NEURAL STATE DYNAMICS ANALYSIS RESULTS ===

TABLE 1: MAIN EFFECTS AND INTERACTIONS
----------------------------------------
                            Feature  Coefficient  Std. Error  P(Effect > 0)  Bayes Factor    Evidence Category  Odds Ratio  OR Lower  OR Upper
                    arthur_speaking        0.214       0.059          1.000        729.83     Extreme evidence       1.239     1.104     1.391
                           has_verb        0.150       0.057          0.996         33.65 Very strong evidence       1.162     1.040     1.298
          has_adj:Group Interaction       -0.142       0.063          0.012         12.60      Strong evidence       0.867     0.766     0.982
       arthur_adj:Group Interaction       -0.166       0.108          0.063          3.24    Moderate evidence       0.847     0.685     1.047
                      lee_girl_verb        0.210       0.149          0.921          2.71   Anecdotal evidence       1.234     0.922     1.652
lee_girl_toge

/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


{'main_effects':                                 Feature  Coefficient  Std. Error  \
 8                       arthur_speaking        0.214       0.059   
 2                              has_verb        0.150       0.057   
 11            has_adj:Group Interaction       -0.142       0.063   
 19         arthur_adj:Group Interaction       -0.166       0.108   
 16                        lee_girl_verb        0.210       0.149   
 1   lee_girl_together:Group Interaction        0.156       0.131   
 0                     lee_girl_together        0.151       0.131   
 18                           arthur_adj        0.104       0.108   
 17      lee_girl_verb:Group Interaction        0.136       0.149   
 7       girl_speaking:Group Interaction       -0.107       0.151   
 10                              has_adj       -0.040       0.063   
 14                             has_noun       -0.033       0.055   
 9     arthur_speaking:Group Interaction        0.033       0.059   
 3            has_

### Cluster 2

In [6]:
cluster_id = 2
model_id = 12
file = os.path.join(brain_glmm_dir, f"cluster{cluster_id}_combined_{model_id}states_deviation_th085", "complete_results.npz")
results = np.load(file, allow_pickle=True)["results"]
main(results.item())


=== NEURAL STATE DYNAMICS ANALYSIS RESULTS ===

TABLE 1: MAIN EFFECTS AND INTERACTIONS
----------------------------------------
                            Feature  Coefficient  Std. Error  P(Effect > 0)  Bayes Factor    Evidence Category  Odds Ratio  OR Lower  OR Upper
                    arthur_speaking        0.500       0.082          1.000  132803938.28     Extreme evidence       1.648     1.404     1.934
                      girl_speaking       -0.946       0.315          0.001         89.69 Very strong evidence       0.388     0.209     0.721
                            has_adj       -0.253       0.093          0.003         39.64 Very strong evidence       0.776     0.647     0.932
                       lee_speaking       -0.211       0.090          0.010         15.21      Strong evidence       0.810     0.678     0.967
       arthur_adj:Group Interaction       -0.269       0.151          0.037          4.94    Moderate evidence       0.764     0.568     1.026
             

/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


{'main_effects':                                 Feature  Coefficient  Std. Error  \
 8                       arthur_speaking        0.500       0.082   
 6                         girl_speaking       -0.946       0.315   
 10                              has_adj       -0.253       0.093   
 4                          lee_speaking       -0.211       0.090   
 19         arthur_adj:Group Interaction       -0.269       0.151   
 14                             has_noun        0.097       0.081   
 16                        lee_girl_verb        0.211       0.214   
 13            has_adv:Group Interaction        0.084       0.092   
 2                              has_verb       -0.061       0.081   
 0                     lee_girl_together        0.140       0.196   
 11            has_adj:Group Interaction       -0.066       0.093   
 18                           arthur_adj       -0.102       0.151   
 15           has_noun:Group Interaction        0.044       0.080   
 1   lee_girl_toge

### Cluster 3

In [9]:
cluster_id = 3
model_id = 2
file = os.path.join(brain_glmm_dir, f"cluster{cluster_id}_combined_{model_id}states_deviation_th085", "complete_results.npz")
results = np.load(file, allow_pickle=True)["results"]
main(results.item())


=== NEURAL STATE DYNAMICS ANALYSIS RESULTS ===

TABLE 1: MAIN EFFECTS AND INTERACTIONS
----------------------------------------
                            Feature  Coefficient  Std. Error  P(Effect > 0)  Bayes Factor  Evidence Category  Odds Ratio  OR Lower  OR Upper
                    arthur_speaking       -0.247       0.059          0.000       6530.16   Extreme evidence       0.781     0.696     0.877
                      lee_girl_verb        0.374       0.146          0.995         26.89    Strong evidence       1.454     1.092     1.936
                  lee_girl_together        0.297       0.130          0.989         13.67    Strong evidence       1.346     1.043     1.737
          has_adj:Group Interaction        0.142       0.063          0.988         12.79    Strong evidence       1.152     1.019     1.304
                            has_adj        0.133       0.063          0.982          9.21  Moderate evidence       1.142     1.009     1.292
                         

/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


{'main_effects':                                 Feature  Coefficient  Std. Error  \
 8                       arthur_speaking       -0.247       0.059   
 16                        lee_girl_verb        0.374       0.146   
 0                     lee_girl_together        0.297       0.130   
 11            has_adj:Group Interaction        0.142       0.063   
 10                              has_adj        0.133       0.063   
 2                              has_verb       -0.114       0.056   
 19         arthur_adj:Group Interaction        0.164       0.108   
 14                             has_noun        0.075       0.055   
 18                           arthur_adj       -0.127       0.108   
 1   lee_girl_together:Group Interaction       -0.148       0.130   
 17      lee_girl_verb:Group Interaction       -0.128       0.145   
 7       girl_speaking:Group Interaction        0.107       0.150   
 9     arthur_speaking:Group Interaction       -0.034       0.059   
 3            has_

### Cluster 4

In [8]:
cluster_id = 4
model_id = 8
file = os.path.join(brain_glmm_dir, f"cluster{cluster_id}_combined_{model_id}states_deviation_th085", "complete_results.npz")
results = np.load(file, allow_pickle=True)["results"]
main(results.item())


=== NEURAL STATE DYNAMICS ANALYSIS RESULTS ===

TABLE 1: MAIN EFFECTS AND INTERACTIONS
----------------------------------------
                            Feature  Coefficient  Std. Error  P(Effect > 0)  Bayes Factor  Evidence Category  Odds Ratio  OR Lower  OR Upper
                    arthur_speaking       -0.371       0.084          0.000      17990.23   Extreme evidence       0.690     0.586     0.813
         has_noun:Group Interaction        0.132       0.075          0.960          4.67  Moderate evidence       1.142     0.985     1.324
  arthur_speaking:Group Interaction       -0.142       0.084          0.044          4.25  Moderate evidence       0.867     0.736     1.022
                           has_noun       -0.094       0.075          0.107          2.17 Anecdotal evidence       0.910     0.785     1.055
          has_adv:Group Interaction       -0.104       0.086          0.113          2.08 Anecdotal evidence       0.901     0.761     1.067
          has_adj:Group I

/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/nfs/home2/yibei/micromamba/envs/prettymouth/lib/python3.11/site-packages/seaborn/matrix.py:207: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


{'main_effects':                                 Feature  Coefficient  Std. Error  \
 8                       arthur_speaking       -0.371       0.084   
 15           has_noun:Group Interaction        0.132       0.075   
 9     arthur_speaking:Group Interaction       -0.142       0.084   
 14                             has_noun       -0.094       0.075   
 13            has_adv:Group Interaction       -0.104       0.086   
 11            has_adj:Group Interaction        0.103       0.087   
 2                              has_verb       -0.088       0.076   
 0                     lee_girl_together       -0.211       0.188   
 10                              has_adj        0.086       0.087   
 6                         girl_speaking       -0.202       0.205   
 19         arthur_adj:Group Interaction       -0.138       0.155   
 5        lee_speaking:Group Interaction        0.058       0.080   
 7       girl_speaking:Group Interaction       -0.140       0.205   
 4                